In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline as imb_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

data = pd.read_csv("healthcare-dataset-stroke-data.csv")

balanced_data = data.dropna()
balanced_data = balanced_data.drop(columns= "id")
balanced_data = balanced_data[balanced_data.age > 20]
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["gender"] == "Other"])
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["work_type"] == "Never_worked"])

features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', "bmi", "smoking_status"]
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']
num_columns = ['age', 'avg_glucose_level', 'bmi']

bal_data_train, bal_data_test = train_test_split(balanced_data, train_size = 0.8, random_state=1, stratify= balanced_data["stroke"])
bal_data_train, bal_data_val = train_test_split(bal_data_train, train_size = 0.8, random_state=1)

X_train = bal_data_train.drop('stroke', axis = 1)
y_train = bal_data_train['stroke']

X_val = bal_data_val.drop('stroke', axis = 1)
y_val = bal_data_val['stroke']

X_test = bal_data_test.drop('stroke', axis = 1)
y_test = bal_data_test['stroke']

In [2]:
preprocessing = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(sparse_output=False), cat_columns),
    ('scaler', StandardScaler(), num_columns)
])

In [3]:
X_train2 = preprocessing.fit_transform(X_train)
X_val2 = preprocessing.transform(X_val)
X_test2 = preprocessing.transform(X_test)

Initial model starts at 0.07 recall

In [5]:
import tensorflow as tf

tf.random.set_seed(1)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=20))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(1, "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['Recall'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               10752     
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 273921 (1.04 MB)
Trainable params: 273921 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model_history = model.fit(X_train2, y_train, validation_data=(X_val2, y_val), epochs=10)

Epoch 1/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1306 - recall: 0.1094 - val_loss: 0.2694 - val_recall: 0.0526
Epoch 2/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1289 - recall: 0.1250 - val_loss: 0.2716 - val_recall: 0.0526
Epoch 3/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1275 - recall: 0.1797 - val_loss: 0.2624 - val_recall: 0.1579
Epoch 4/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1205 - recall: 0.2031 - val_loss: 0.2783 - val_recall: 0.0789
Epoch 5/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1147 - recall: 0.2188 - val_loss: 0.3003 - val_recall: 0.0789
Epoch 6/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1093 - recall: 0.2500 - val_loss: 0.3154 - val_recall: 0.0526
Epoch 7/10
79/79 [==============================] - 0s 3ms/step - loss: 0.1063 - recall: 0.2969 - val_loss: 0.3403 - val_recall: 0.0789
Epoch 8/10
79/79 [==============================

In [8]:
model.evaluate(X_test2, y_test)

25/25 [==============================] - 0s 1ms/step - loss: 0.4169 - recall: 0.0476


[0.4168650805950165, 0.0476190485060215]

https://imbalanced-learn.org/stable/miscellaneous.html

https://www.freecodecamp.org/news/binary-classification-made-simple-with-tensorflow/
